# GLM analysis on the main tasks
It uses nilearn and performs the following steps:
1. Load the data from fmriPrep in BIDS format
2. Iterate on the subjects to:
   1. Select the predictors and confounds for the design matrix
   2. Generate 1st level model
   3. Estimate contrast maps
3. Generate group level maps
4. Generate hMT+ mask

In [ ]:
# Imports
import os
import glob
from nilearn.glm.first_level import first_level_from_bids
from nilearn.interfaces.bids import save_glm_to_bids
from nilearn.glm import threshold_stats_img
from nilearn import plotting
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from nilearn.glm.second_level import SecondLevelModel
from nilearn.reporting import get_clusters_table
from nilearn.image import math_img
from nilearn.masking import apply_mask

In [ ]:
# Settings
data_dir = '/DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK1/'
space_label = "MNI152NLin2009cAsym"
derivatives_folder = "derivatives/fmriprep/fmriprep"
task_label = "01"
smoothing_fwhm = 4.0
high_pass_hz = 0.007
out_dir = os.path.join(data_dir,"derivatives","nilearn_glm")

## 1. Load the data from fmriPrep in BIDS format

In [ ]:
# import first level data automatically from fmriPrep derivatives
(
    models,
    models_run_imgs,
    models_events,
    models_confounds,
) = first_level_from_bids(
    data_dir,
    task_label,
    space_label,
    hrf_model="spm",
    noise_model="ar2",
    smoothing_fwhm=smoothing_fwhm,
    high_pass=high_pass_hz,
    slice_time_ref=None,
    n_jobs=8,
    minimize_memory = False,
    derivatives_folder=derivatives_folder,
)

In [ ]:
# edit all models_events to exclude the conditions that end with '_p'
for ii in range(len(models_events)):
    for jj in range(len(models_events[ii])):
        models_events[ii][jj] = models_events[ii][jj][models_events[ii][jj].trial_type.str.endswith('_p') == False]

models_events[0][0]

# 2. Deal with contrast names
These should be balanced here.

In [ ]:
# condition names
condition_names = ['Q1_db','Q2_db','Q3_db','Q4_db']

# create strings for contrasts in the format of "condition_name - Noise"
contrasts = []

# add contrast all conditions vs. noise
contrasts.append("Q1_db + Q2_db + Q3_db + Q4_db - Noise*4")

# iterate to add the other contrasts
for condition in condition_names:
    contrasts.append(condition + " - Noise")

# add more contrasts based on the arousal and valence
contrasts.append("Q1_db*0.5 + Q2_db*0.5 - Q3_db*0.5 - Q4_db*0.5") # positive arousal vs. negative arousal
contrasts.append("Q1_db*0.5 + Q4_db*0.5 - Q3_db*0.5 - Q2_db*0.5") # positive valence vs. negative valence

contrasts

In [ ]:
# This is to use the contrasts as names for the output files
contrasts_renamed = ['Q1234MinusNoise',
                     'Q1MinusNoise',
                     'Q2MinusNoise',
                     'Q3MinusNoise',
                     'Q4MinusNoise',
                     'PositiveArousalMinusNegativeArousal',
                     'PositiveValenceMinusNegativeValence']

contrasts_renamed

# 3. Iterate on the subjects
This was computed in the INCD cluster for speed :) It takes a while, as it estimated the multi-run GLM for each subject.

In [ ]:
for idx in range(len(models)):

    # fetch model
    model, imgs, events, confounds = (
        models[idx],
        models_run_imgs[idx],
        models_events[idx],
        models_confounds[idx],
    )

    subject = f"sub-{model.subject_label}"

    print(f"Computing 1st level model for subject: {subject}")

    # trim confounds and replace NaNs with 0
    for ii in range(len(confounds)):
        confounds[ii] = confounds[ii][['trans_x', 'trans_x_derivative1', 'trans_x_power2', 'trans_x_derivative1_power2',
                                        'trans_y', 'trans_y_derivative1', 'trans_y_power2', 'trans_y_derivative1_power2',
                                        'trans_z', 'trans_z_derivative1', 'trans_z_power2', 'trans_z_derivative1_power2',
                                        'rot_x', 'rot_x_derivative1', 'rot_x_power2', 'rot_x_derivative1_power2',
                                        'rot_y', 'rot_y_derivative1', 'rot_y_power2', 'rot_y_derivative1_power2',
                                        'rot_z', 'rot_z_derivative1', 'rot_z_power2', 'rot_z_derivative1_power2',
                                            ]]
    
        confounds[ii] = confounds[ii].fillna(0)
    
    # Fit and contrasts
    model.fit(imgs, events, confounds)

    # create and save z_map, t_map, and beta_map to nifti files for every contrast
    for ii in range(len(contrasts)):

        z_map = model.compute_contrast(contrasts[ii], output_type="z_score")
        t_map = model.compute_contrast(contrasts[ii], output_type="stat")
        beta_map = model.compute_contrast(contrasts[ii], output_type="effect_size")

        z_map.to_filename(os.path.join(out_dir, f"{subject}_task-{task_label}_stat-z_con-{contrasts_renamed[ii]}.nii.gz"))
        t_map.to_filename(os.path.join(out_dir, f"{subject}_task-{task_label}_stat-t_con-{contrasts_renamed[ii]}.nii.gz"))
        beta_map.to_filename(os.path.join(out_dir, f"{subject}_task-{task_label}_stat-beta_con-{contrasts_renamed[ii]}.nii.gz"))

        # create figure with thresholded map for fun
        clean_map, threshold = threshold_stats_img(
            z_map, alpha=0.05, height_control="bonferroni", cluster_threshold=25
        )

        plotting.plot_glass_brain(
            clean_map,
            colorbar=True,
            threshold=threshold,
            plot_abs=False,
            display_mode="ortho",
            figure=plt.figure(figsize=(10, 4)),
        )

        plt.savefig(os.path.join(
            out_dir,
            f"{subject}_task-{task_label}_plot-z_con-{contrasts_renamed[ii]}_c-bonferroni_p-0.05_clusterk-25.png"
            )
        )

        # Export cluster table
        table = get_clusters_table(z_map, threshold, 25)
        table.to_csv(os.path.join(
            out_dir,
            f"{subject}_task-{task_label}_table-clusters_con-{contrasts_renamed[ii]}_c-bonferroni_p-0.05_clusterk-25.tsv"
            ),
            sep='\t'
        )

    # Attempt to free memory
    del model, imgs, events, confounds
    

# 4. Group level analysis

In [ ]:
# just some paths again
out_dir = os.path.join(data_dir,"derivatives","nilearn_glm")
out_dir_group = os.path.join(data_dir,"derivatives","nilearn_glm","group")
if not os.path.exists(out_dir_group):
    os.makedirs(out_dir_group)

In [ ]:
# choose a contrast
c_idx = 0

# List all tmap nii.gz files
tmap_files = glob.glob(
    os.path.join(out_dir,
        f"sub-*_task-{task_label}_stat-t_con-{contrasts_renamed[c_idx]}.nii.gz"
    )
)
tmap_files.sort()

# List all zmap nii.gz files
zmap_files = glob.glob(
    os.path.join(out_dir,
        f"sub-*_task-{task_label}_stat-z_con-{contrasts_renamed[c_idx]}.nii.gz"
    )
)
zmap_files.sort()

subject_list = [os.path.basename(f).split('_')[0] for f in tmap_files]
subject_list

In [ ]:
#| label: plot_glass_matrix_singlesubject

# Plot all subjects t_maps for a given contrast
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(10, 10))

for cidx, tmap in enumerate(tmap_files):
    P = plotting.plot_glass_brain(
        tmap,
        colorbar=True,
        threshold=6.0,
        vmax=25,
        axes=axes[cidx % 4, int(cidx / 4)],
        plot_abs=False,
        display_mode="x",
    )
    P.title(subject_list[cidx], size=8)

fig.suptitle(f"subjects t_map for contrast {contrasts[c_idx]}")
plt.show()

In [ ]:
# create design matrix for 2nd level
second_level_input = zmap_files
design_matrix_g = pd.DataFrame(
    [1] * len(second_level_input),
    columns=["intercept"],
)

# define 2nd level model
second_level_model = SecondLevelModel(smoothing_fwhm=6.0, n_jobs=12)
second_level_model.minimize_memory = False
second_level_model = second_level_model.fit(
    second_level_input,
    design_matrix=design_matrix_g,
)

# compute contrast (z score map)
z_map_g = second_level_model.compute_contrast(
    second_level_contrast="intercept",
    output_type="z_score",
)

# compute contrast (t score map)
t_map_g = second_level_model.compute_contrast(
    second_level_contrast="intercept",
    output_type="stat",
)

# compute contrast (beta map)
beta_map_g = second_level_model.compute_contrast(
    second_level_contrast="intercept",
    output_type='effect_size',
)

In [ ]:
# Threshold zmap and plot it
hc = None # None, 'bonferroni', 'fdr'
ct = 25 # cluster threshold
alpha = 0.001 # p-value threshold

clean_map_g, threshold_g = threshold_stats_img(
    z_map_g, alpha=alpha, height_control=hc, cluster_threshold=ct
)

plotting.plot_glass_brain(
    clean_map_g,
    colorbar=True,
    threshold=threshold_g,
    plot_abs=False,
    display_mode="ortho",
    vmax=8,
    figure=plt.figure(figsize=(10, 4)),
    symmetric_cbar=False,
    cmap=nilearn_cmaps["cold_hot"],
)

plt.savefig(os.path.join(out_dir_group,
                         f"group_task-{task_label}_plot-z_con-_{contrasts_renamed[c_idx]}_c-{hc}_p-{alpha}_clusterk-{ct}.png"))

In [ ]:
# Export cluster table
table_g,cluster_map_g = get_clusters_table(z_map_g, threshold_g, ct,
                                return_label_maps=True)

table_g.to_csv(os.path.join(out_dir_group,
                          f"group_task-{task_label}_table-clusters_con-{contrasts_renamed[c_idx]}_c-{hc}_p-0.05_clusterk-{ct}.tsv"),sep='\t')
#print(table)
#print(table.to_latex())
table_g

In [ ]:
# View map interactively
plotting.view_img(clean_map_g,
         threshold=threshold_g
        )

# ROI analysis

In [ ]:
#| label: loc_group_interactive
# Show cluster_map_g
plotting.view_img(cluster_map_g[0],
                  vmax=3, vmin=0,
                  resampling_interpolation='nearest',
                  cmap='hot',
                  symmetric_cmap=False)

In [ ]:
# find unique values in cluster_map_g[0]
np.unique(cluster_map_g[0].get_fdata())

In [ ]:
# Generate auditory cluster mask as an example
# in this case, it is values 1 and 2 from cluster_map_g
aux_mask1 = math_img('img == 3', img=cluster_map_g[0])
aux_mask2 = math_img('img == 5', img=cluster_map_g[0])
mask_hMT = math_img('img1 + img2', img1=aux_mask1, img2=aux_mask2)

plotting.view_img(mask_hMT, vmax=1, vmin=0, symmetric_cmap=False, cmap='hot')

# save mask
#mask_hMT.to_filename(os.path.join(data_dir,"derivatives","nilearn_glm","group",'mask_hMT.nii.gz'))


In [ ]:
# fetch map values inside mask_hMT  
# Apply mask to z_map_g
z_map_hMT = apply_mask(z_map_g, mask_hMT)

# Apply mask to beta_map_g
beta_map_hMT = apply_mask(beta_map_g, mask_hMT)

# Estimate mean of z_map_hMT
z_map_hMT_mean = np.mean(z_map_hMT)

z_map_hMT_mean